In [1]:
from langchain_ollama import OllamaEmbeddings
from langgraph.store.memory import InMemoryStore

# Create store with semantic search enabled
embeddings = OllamaEmbeddings(model="snowflake-arctic-embed2")
store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
    }
)

In [2]:
store.put(("user_123", "memories"), "1", {"text": "I love pizza"})
store.put(("user_123", "memories"), "2", {"text": "I prefer Italian food"})
store.put(("user_123", "memories"), "3", {"text": "I don't like spicy food"})
store.put(("user_123", "memories"), "3", {"text": "I am studying econometrics"})
store.put(("user_123", "memories"), "3", {"text": "I am a plumber"})

In [3]:
# Find memories about food preferences
memories = store.search(("user_123", "memories"), query="I like food?", limit=5)

for memory in memories:
    print(f'Memory: {memory.value["text"]} (similarity: {memory.score})')

Memory: I prefer Italian food (similarity: 0.5678727741490065)
Memory: I love pizza (similarity: 0.5217003208170932)
Memory: I am a plumber (similarity: 0.38765420504835063)


In [4]:
from typing import Optional

from langchain_ollama import ChatOllama
from langgraph.store.base import BaseStore
from langgraph.graph import START, MessagesState, StateGraph


model_name = 'llama3.2'
llm = ChatOllama(model=model_name, temperature=0.9)


def chat(state, *, store: BaseStore):
    # Search based on user's last message
    items = store.search(
        ("user_123", "memories"), query=state["messages"][-1].content, limit=2
    )
    memories = "\n".join(item.value["text"] for item in items)
    memories = f"## Memories of user\n{memories}" if memories else ""
    response = llm.invoke(
        [
            {"role": "system", "content": f"You are a helpful assistant.\n{memories}"},
            *state["messages"],
        ]
    )
    return {"messages": [response]}


builder = StateGraph(MessagesState)
builder.add_node(chat)
builder.add_edge(START, "chat")
graph = builder.compile(store=store)

for message, metadata in graph.stream(
    input={"messages": [{"role": "user", "content": "I'm hungry"}]},
    stream_mode="messages",
):
    print(message.content, end="")

It sounds like it's time for lunch or dinner! Since you mentioned you prefer Italian food earlier, would you like some recommendations for Italian dishes or maybe even some pizza ideas?